In [1]:
import pandas as pd
import numpy as np
import glob

from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split

In [2]:
import os
import glob

# Путь к папке с данными
data_dir = "../data"

# Шаблон для поиска файлов в папке data
file_pattern = os.path.join(data_dir, "*.csv")  # Ищем все CSV файлы в папке data

# Поиск файлов
file_paths = glob.glob(file_pattern)

# Вывод найденных файлов
print("Найденные файлы:", file_paths)

Найденные файлы: ['../data\\APTUSDT_hourly_data.csv', '../data\\BNBUSDT_hourly_data.csv', '../data\\BTCUSDT_hourly_data.csv', '../data\\CAKEUSDT_hourly_data.csv', '../data\\data_compare_eda.csv', '../data\\ETHUSDT_hourly_data.csv', '../data\\HBARUSDT_hourly_data.csv', '../data\\JUPUSDT_hourly_data.csv', '../data\\LDOUSDT_hourly_data.csv', '../data\\LTCUSDT_hourly_data.csv', '../data\\PEPEUSDT_hourly_data.csv', '../data\\SOLUSDT_hourly_data.csv', '../data\\STRKUSDT_hourly_data.csv', '../data\\TONUSDT_hourly_data.csv', '../data\\USDCUSDT_hourly_data.csv', '../data\\XRPUSDT_hourly_data.csv']


In [3]:
import pandas as pd
import glob
import os

# Путь к папке с данными
data_dir = "../data"

# Создаем пустой список для хранения данных
data_frames = []

# Изменяем шаблон поиска: только файлы вида <tic><pair>_hourly_data.csv
file_pattern = os.path.join(data_dir, "*_hourly_data.csv")
file_paths = glob.glob(file_pattern)

# Обрабатываем каждый подходящий файл
for file_path in file_paths:
    # Имя файла без пути
    file_name = os.path.basename(file_path)

    # Извлекаем тикер и пару (например, "APTUSDT" из "APTUSDT_hourly_data.csv")
    symbol = file_name.replace("_hourly_data.csv", "")  # Получаем "APTUSDT"

    try:
        # Читаем CSV-файл
        df = pd.read_csv(file_path)

        # Очищаем имена столбцов
        df.columns = df.columns.str.strip().str.lower()

        # Переименовываем столбцы
        df.rename(columns={
            "open time": "date",
            "open": "open",
            "high": "high",
            "low": "low",
            "close": "close",
            "volume": "volume"
        }, inplace=True)

        # Добавляем столбец tic (теперь с полным значением, например APTUSDT)
        df["tic"] = symbol

        # Проверяем наличие необходимых столбцов
        required_columns = ["date", "open", "high", "low", "close", "volume"]
        if all(col in df.columns for col in required_columns):
            df = df[required_columns + ["tic"]]
        else:
            print(f"В файле {file_path} отсутствуют необходимые столбцы.")
            print("Доступные столбцы:", df.columns)
            continue

        # Добавляем DataFrame в список
        data_frames.append(df)

    except Exception as e:
        print(f"Ошибка при обработке файла {file_path}: {e}")

# Объединяем все DataFrame в один
if data_frames:
    final_df = pd.concat(data_frames, ignore_index=True)

    # Преобразуем столбец date в datetime
    final_df["date"] = pd.to_datetime(final_df["date"])

    # Выводим первые строки итогового DataFrame
    print(final_df.head())
    print("Размер итогового DataFrame:", final_df.shape)
else:
    print("Нет подходящих данных для объединения.")

                 date    open      high     low   close       volume      tic
0 2022-10-19 01:00:00  1.0000  100.0000  1.0000  8.4946  14808472.50  APTUSDT
1 2022-10-19 02:00:00  8.5077    8.5895  6.9170  7.4817   6936252.07  APTUSDT
2 2022-10-19 03:00:00  7.4817    7.6000  6.6000  6.8147   3917949.82  APTUSDT
3 2022-10-19 04:00:00  6.8038    7.5300  6.7456  7.1741   3598717.51  APTUSDT
4 2022-10-19 05:00:00  7.1806    8.2500  7.1770  7.9068   5028746.51  APTUSDT
Размер итогового DataFrame: (430665, 7)


In [4]:
final_df['tic'].unique()

array(['APTUSDT', 'BNBUSDT', 'BTCUSDT', 'CAKEUSDT', 'ETHUSDT', 'HBARUSDT',
       'JUPUSDT', 'LDOUSDT', 'LTCUSDT', 'PEPEUSDT', 'SOLUSDT', 'STRKUSDT',
       'TONUSDT', 'USDCUSDT', 'XRPUSDT'], dtype=object)

In [5]:
symbols = ['JUPUSDT', 'PEPEUSDT', 'APTUSDT', 'CAKEUSDT', 'HBARUSDT', 'STRKUSDT', 'USDCUSDT']

In [6]:
df_prep = final_df[final_df['tic'].isin(symbols)]

max_start_date = df_prep.groupby('tic')['date'].min().max()

df_prep_aligned = df_prep[df_prep['date'] >= max_start_date]

print("Aligned start date:", max_start_date)
print(df_prep_aligned.groupby('tic')['date'].min())  # Должны быть одинаковые даты для всех тикеров
print(df_prep_aligned.shape)

Aligned start date: 2024-02-20 13:00:00
tic
APTUSDT    2024-02-20 13:00:00
CAKEUSDT   2024-02-20 13:00:00
HBARUSDT   2024-02-20 13:00:00
JUPUSDT    2024-02-20 13:00:00
PEPEUSDT   2024-02-20 13:00:00
STRKUSDT   2024-02-20 13:00:00
USDCUSDT   2024-02-20 13:00:00
Name: date, dtype: datetime64[ns]
(73619, 7)


In [7]:
df_prep_aligned

,date,open,high,low,close,volume,tic
11747,2024-02-20 13:00:00,9.7611,9.9990,9.7409,9.9920,354607.62,APTUSDT
11748,2024-02-20 14:00:00,9.9905,10.0277,9.5872,9.6784,550845.81,APTUSDT
11749,2024-02-20 15:00:00,9.6815,9.6990,9.0609,9.4612,915566.12,APTUSDT
11750,2024-02-20 16:00:00,9.4600,9.5253,9.2789,9.2950,391945.24,APTUSDT
11751,2024-02-20 17:00:00,9.2952,9.3905,9.1959,9.3817,324567.49,APTUSDT
...,...,...,...,...,...,...,...
392664,2025-05-03 13:00:00,0.9997,0.9998,0.9997,0.9997,6114367.00,USDCUSDT
392665,2025-05-03 14:00:00,0.9998,0.9998,0.9997,0.9998,9250809.00,USDCUSDT
392666,2025-05-03 15:00:00,0.9998,0.9998,0.9997,0.9997,6475349.00,USDCUSDT
392667,2025-05-03 16:00:00,0.9997,0.9998,0.9997,0.9997,2906779.00,USDCUSDT


## Preprocess Data

In [8]:
import pandas as pd
import ta

# 1. Убедись, что дата отсортирована
df_prep_aligned = df_prep_aligned.sort_values(by=['tic', 'date']).reset_index(drop=True)

# 2. Функция для добавления индикаторов к группе по tic
def add_technical_features(group):
    # Добавляем все доступные индикаторы через ta
    group = ta.add_all_ta_features(
        group,
        open="open",
        high="high",
        low="low",
        close="close",
        volume="volume",
        fillna=True
    )

    # Вручную добавляем SMA(60), чтобы быть уверенным
    group['close_60_sma'] = group['close'].rolling(window=60).mean()

    return group

# 3. Применяем к каждому тикеру отдельно
df = df_prep_aligned.groupby("tic", group_keys=False).apply(add_technical_features)

# 4. Словарь: исходное имя → новое имя (удобное для RL)
indicator_mapping = {
    # --- MACD ---
    'trend_macd': 'macd',
    'trend_macd_signal': 'macd_signal',
    'trend_macd_diff': 'macd_diff',

    # --- Bollinger Bands ---
    'volatility_bbh': 'boll_ub',
    'volatility_bbl': 'boll_lb',
    'volatility_bbm': 'boll_mb',

    # --- RSI ---
    'momentum_rsi': 'rsi_30',

    # --- CCI ---
    'trend_cci': 'cci_30',

    # --- ADX / DX ---
    'trend_adx': 'dx_30',
    'trend_adx_pos': 'adx_plus',
    'trend_adx_neg': 'adx_minus',

    # --- Stochastic RSI ---
    'momentum_stoch_rsi_k': 'stoch_rsi_k',
    'momentum_stoch_rsi_d': 'stoch_rsi_d',

    # --- OBV ---
    'volume_obv': 'obv',

    # --- ATR ---
    'volatility_atr': 'atr',

    # --- Vortex Indicator ---
    'trend_vortex_ind_pos': 'vortex_up',
    'trend_vortex_ind_neg': 'vortex_down',

    # --- Trix ---
    'trend_trix': 'trix',

    # --- Awesome Oscillator ---
    'momentum_ao': 'ao',

    # --- Parabolic SAR ---
    'trend_psar_up_indicator': 'psar_up',
    'trend_psar_down_indicator': 'psar_down',

    # --- Rate of Change ---
    'momentum_roc': 'roc',

    # --- SMA ---
    'trend_sma_fast': 'close_15_sma',
    'trend_sma_slow': 'close_30_sma',
}

# Проверяем, какие колонки действительно есть в данных
available_columns = df.columns.tolist()
valid_mapping = {src: dst for src, dst in indicator_mapping.items() if src in available_columns}

# Переименовываем только существующие колонки
df.rename(columns=valid_mapping, inplace=True)

# 5. Добавляем вручную close_60_sma (гарантировано!)
df['close_60_sma'] = df.groupby('tic')['close'].transform(lambda x: x.rolling(window=60).mean())

# 6. Проверяем результат
print("Все доступные столбцы:")
print(df.columns.tolist())

print("\nПример данных:")
print(df[['date', 'tic', 'close', 
          'macd', 'macd_diff', 
          'boll_ub', 'boll_lb', 'boll_mb', 
          'rsi_30', 'cci_30', 
          'close_30_sma', 'close_60_sma']].head(60))

d:\__projects__\diploma\portfolios-optimization\.venv\lib\site-packages\ta\utils.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(value)
d:\__projects__\diploma\portfolios-optimization\.venv\lib\site-packages\ta\utils.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(value)
d:\__projects__\diploma\portfolios-optimization\.venv\lib\site-packages\ta\utils.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(value)
d:\__projects__\diploma\portfolios-optimization\.venv\lib\site-packages\ta\utils.py:35: FutureWarning: Series.fillna with 'method' is deprecated and wi

Все доступные столбцы:
['date', 'open', 'high', 'low', 'close', 'volume', 'tic', 'volume_adi', 'obv', 'volume_cmf', 'volume_fi', 'volume_em', 'volume_sma_em', 'volume_vpt', 'volume_vwap', 'volume_mfi', 'volume_nvi', 'boll_mb', 'boll_ub', 'boll_lb', 'volatility_bbw', 'volatility_bbp', 'volatility_bbhi', 'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl', 'volatility_kcw', 'volatility_kcp', 'volatility_kchi', 'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'volatility_dcm', 'volatility_dcw', 'volatility_dcp', 'atr', 'volatility_ui', 'macd', 'macd_signal', 'macd_diff', 'close_15_sma', 'close_30_sma', 'trend_ema_fast', 'trend_ema_slow', 'vortex_up', 'vortex_down', 'trend_vortex_ind_diff', 'trix', 'trend_mass_index', 'trend_dpo', 'trend_kst', 'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_conv', 'trend_ichimoku_base', 'trend_ichimoku_a', 'trend_ichimoku_b', 'trend_stc', 'dx_30', 'adx_plus', 'adx_minus', 'cci_30', 'trend_visual_ichimoku_a', 'trend_visual_ichim

d:\__projects__\diploma\portfolios-optimization\.venv\lib\site-packages\ta\utils.py:33: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(method='bfill')
d:\__projects__\diploma\portfolios-optimization\.venv\lib\site-packages\ta\utils.py:33: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(method='bfill')
d:\__projects__\diploma\portfolios-optimization\.venv\lib\site-packages\ta\utils.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(value)
d:\__projects__\diploma\portfolios-optimization\.venv\lib\site-packages\ta\utils.py:35: FutureWarning: Series.fillna with 'method' is

In [9]:
# fe = FeatureEngineer(
#                     use_technical_indicator=True,
#                     use_turbulence=False,
#                     user_defined_feature = False)

# df = fe.preprocess_data(df_prep_aligned)
# print(df.shape)
# print(df['tic'].unique())

In [10]:
# Add covariance matrix as states
df = df.sort_values(['date','tic'], ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back
lb_val = 24
lookback = lb_val
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback = data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values 
  cov_list.append(covs)
  
df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)
print(df.shape)
df.head()

(73451, 96)


,date,open,high,low,close,volume,tic,volume_adi,obv,volume_cmf,...,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr,close_60_sma,cov_list,return_list
0,2024-02-21 13:00:00,9.278200,9.300000,9.146400,9.203700,1.585345e+05,APTUSDT,-6.873148e+05,-2.545573e+06,-0.197577,...,-13.845722,-12.320601,-1.525120,9.253141,-0.815786,-0.819131,-7.889311,NaN,"[[0.00013639724644876403, 0.000106712407983733...",tic APTUSDT CAKEUSDT HBARU...
1,2024-02-21 13:00:00,2.720000,2.720000,2.686000,2.703000,3.406541e+05,CAKEUSDT,-1.400960e+04,-1.806891e+06,0.049516,...,7.159000,-3.758031,10.917031,2.712555,-0.661521,-0.663719,-4.487633,NaN,"[[0.00013639724644876403, 0.000106712407983733...",tic APTUSDT CAKEUSDT HBARU...
2,2024-02-21 13:00:00,0.104200,0.105100,0.102300,0.103900,2.645657e+07,HBARUSDT,-1.088864e+08,-2.572574e+07,-0.119244,...,-6.929859,-7.149416,0.219558,0.107265,-0.287908,-0.288323,-5.287147,NaN,"[[0.00013639724644876403, 0.000106712407983733...",tic APTUSDT CAKEUSDT HBARU...
3,2024-02-21 13:00:00,0.473300,0.475600,0.465100,0.470300,2.522664e+06,JUPUSDT,5.746169e+06,-2.645261e+07,0.035446,...,-8.789005,-13.747967,4.958962,0.480927,-0.612849,-0.614734,-10.059285,NaN,"[[0.00013639724644876403, 0.000106712407983733...",tic APTUSDT CAKEUSDT HBARU...
4,2024-02-21 13:00:00,0.000001,0.000001,0.000001,0.000001,1.881588e+11,PEPEUSDT,-7.840527e+09,-2.500224e+12,0.018805,...,-10.544451,-9.040798,-1.503653,0.000001,-0.877193,-0.881063,-7.377049,NaN,"[[0.00013639724644876403, 0.000106712407983733...",tic APTUSDT CAKEUSDT HBARU...


## Design Environment

In [11]:
# Определяем границы для обучающей выборки
start_train = df['date'].min()  # Самая ранняя дата в данных
end_train = df['date'].quantile(0.8)  # 80% данных для обучения (по времени)

print("Start of training period:", start_train)
print("End of training period:", end_train)

Start of training period: 2024-02-21 13:00:00
End of training period: 2025-02-05 07:00:00


In [12]:
# Определяем границы для тестовой выборки
start_test = end_train  # Начало тестовой выборки = конец обучающей
end_test = df['date'].max()  # Самая поздняя дата в данных

print("Start of testing period:", start_test)
print("End of testing period:", end_test)

Start of testing period: 2025-02-05 07:00:00
End of testing period: 2025-05-03 16:00:00


In [25]:
train = data_split(df, start_train, end_train)
train.head()

,date,open,high,low,close,volume,tic,volume_adi,obv,volume_cmf,...,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr,close_60_sma,cov_list,return_list
0,2024-02-21 13:00:00,9.278200,9.300000,9.146400,9.203700,1.585345e+05,APTUSDT,-6.873148e+05,-2.545573e+06,-0.197577,...,-13.845722,-12.320601,-1.525120,9.253141,-0.815786,-0.819131,-7.889311,NaN,"[[0.00013639724644876403, 0.000106712407983733...",tic APTUSDT CAKEUSDT HBARU...
0,2024-02-21 13:00:00,2.720000,2.720000,2.686000,2.703000,3.406541e+05,CAKEUSDT,-1.400960e+04,-1.806891e+06,0.049516,...,7.159000,-3.758031,10.917031,2.712555,-0.661521,-0.663719,-4.487633,NaN,"[[0.00013639724644876403, 0.000106712407983733...",tic APTUSDT CAKEUSDT HBARU...
0,2024-02-21 13:00:00,0.104200,0.105100,0.102300,0.103900,2.645657e+07,HBARUSDT,-1.088864e+08,-2.572574e+07,-0.119244,...,-6.929859,-7.149416,0.219558,0.107265,-0.287908,-0.288323,-5.287147,NaN,"[[0.00013639724644876403, 0.000106712407983733...",tic APTUSDT CAKEUSDT HBARU...
0,2024-02-21 13:00:00,0.473300,0.475600,0.465100,0.470300,2.522664e+06,JUPUSDT,5.746169e+06,-2.645261e+07,0.035446,...,-8.789005,-13.747967,4.958962,0.480927,-0.612849,-0.614734,-10.059285,NaN,"[[0.00013639724644876403, 0.000106712407983733...",tic APTUSDT CAKEUSDT HBARU...
0,2024-02-21 13:00:00,0.000001,0.000001,0.000001,0.000001,1.881588e+11,PEPEUSDT,-7.840527e+09,-2.500224e+12,0.018805,...,-10.544451,-9.040798,-1.503653,0.000001,-0.877193,-0.881063,-7.377049,NaN,"[[0.00013639724644876403, 0.000106712407983733...",tic APTUSDT CAKEUSDT HBARU...


In [26]:
# Убедись, что имя переменной твоего трейн-датасета - 'train'
# Если имя другое, замени 'train' в коде ниже на правильное имя

print("Filling NaNs in close_60_sma...")
# Заполняем NaN в 'close_60_sma' методом bfill внутри каждой группы 'tic'
train['close_60_sma'] = train.groupby('tic', group_keys=False)['close_60_sma'].bfill()

# Проверяем, остались ли NaN (на всякий случай, если bfill не сработал для каких-то групп)
remaining_nans = train['close_60_sma'].isnull().sum()
if remaining_nans > 0:
    print(f"Warning: {remaining_nans} NaNs still remain in close_60_sma after bfill. Filling with 0.")
    # Если остались NaN после bfill (маловероятно), можно заполнить нулем или ffill
    train['close_60_sma'].fillna(0, inplace=True) # Или .ffill(inplace=True)

# Финальная проверка
print("\\n--- Verification after NaN fill ---")
nan_counts = train.isnull().sum()
print(f"NaN counts per column in train_df AFTER fill:\n{nan_counts[nan_counts > 0]}")
total_nans = nan_counts.sum()
print(f"Total NaNs in train_df AFTER fill: {total_nans}")
if total_nans == 0:
    print("Successfully removed NaNs from train data.")
else:
    print("Warning: NaNs still present in train data.")

print(f"Infinities in train_df: {np.isinf(train.select_dtypes(include=[np.number])).sum().sum()}")
print("------------------------------------\\n")


Filling NaNs in close_60_sma...
\n--- Verification after NaN fill ---
NaN counts per column in train_df AFTER fill:
Series([], dtype: int64)
Total NaNs in train_df AFTER fill: 0
Successfully removed NaNs from train data.
Infinities in train_df: 0
------------------------------------\n


https://www.kaggle.com/code/learnmore1/deep-reinforcement-learning-for-stock-trading-1#Part-5.-Design-Environment

In [27]:
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from gymnasium.utils import seeding
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import DDPG
from finrl.agents.stablebaselines3.models import DRLAgent


class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=24 * 30 * 3,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        # self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,)) 
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,))
        # self.action_space = spaces.Box(low=0, high=1, shape=(self.stock_dim,), dtype=np.float32)
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold        
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        # Check if the episode is done
        self.terminal = self.day >= len(self.df.index.unique()) - 1
    
        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(), 'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()
    
            plt.plot(self.portfolio_return_memory, 'r')
            plt.savefig('results/rewards.png')
            plt.close()
    
            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))
    
            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() != 0:
                sharpe = ((24 * 30 * 3)**0.5) * df_daily_return['daily_return'].mean() / \
                         df_daily_return['daily_return'].std()
                print("Sharpe: ", sharpe)
            print("=================================")
    
            # Return observation, reward, terminated, truncated, and info
            return self.state, self.reward, self.terminal, False, {}
    
        else:
            # Normalize actions
            weights = self.softmax_normalization(actions)
            self.actions_memory.append(weights)
            last_day_memory = self.data
    
            # Load next state
            self.day += 1
            self.data = self.df.loc[self.day, :]
            self.covs = self.data['cov_list'].values[0]
            self.state = np.append(
                np.array(self.covs),
                [self.data[tech].values.tolist() for tech in self.tech_indicator_list],
                axis=0
            )
    
            # Calculate portfolio return
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values) - 1) * weights)
            new_portfolio_value = self.portfolio_value * (1 + portfolio_return)
            self.portfolio_value = new_portfolio_value
    
            # Save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)
    
            # Reward is the new portfolio value
            self.reward = new_portfolio_value
    
            # Return observation, reward, terminated, truncated, and info
            return self.state, self.reward, self.terminal, False, {}

    def reset(self, seed=None, options=None):
        # Если нужно, инициализируйте генератор случайных чисел
        if seed is not None:
            self._seed(seed)
        
        # Остальная логика вашего метода reset()
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day, :]
        self.covs = self.data['cov_list'].values[0]
        self.state = np.append(
            np.array(self.covs),
            [self.data[tech].values.tolist() for tech in self.tech_indicator_list],
            axis=0
        )
        self.portfolio_value = self.initial_amount
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.actions_memory = [[1 / self.stock_dim] * self.stock_dim]
        self.date_memory = [self.data.date.unique()[0]]
        return self.state, {}  # Возвращайте состояние и пустой словарь (info)
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [28]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 7, State Space: 7


In [29]:
from finrl import config
from finrl import config_tickers
# Create Folders
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [30]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.INDICATORS, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


## Implement DRL Algorithms

## Model 1: A2C

In [32]:
from finrl.agents.stablebaselines3.models import DRLAgent
agent = DRLAgent(env = env_train)

In [33]:
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=50000)

trained_a2c.save('./trained_models/trained_a2c.zip')

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cpu device
-------------------------------------
| time/                 |           |
|    fps                | 280       |
|    iterations         | 100       |
|    time_elapsed       | 1         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -9.92     |
|    explained_variance | 2.38e-07  |
|    learning_rate      | 0.0002    |
|    n_updates          | 99        |
|    policy_loss        | 4.85e+07  |
|    reward             | 1880254.4 |
|    std                | 0.998     |
|    value_loss         | 3.62e+13  |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 348       |
|    iterations         | 200       |
|    time_elapsed       | 2         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -9.91     |
|    explain

## Model 2: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=80000)

trained_ppo.save('./trained_models/trained_ppo.zip')

## Model 3: DDPG

In [ ]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

trained_ddpg.save('./trained_models/trained_ddpg.zip')

## Model 3: SAC

In [ ]:
import numpy as np
import pandas as pd
import gymnasium as gym
from stable_baselines3 import SAC 
from finrl.agents.stablebaselines3.models import DRLAgent


# Параметры, специфичные для ПОЛИТИКИ (архитектура сети)
POLICY_KWARGS = dict(net_arch=[256, 256])

# ОСНОВНЫЕ параметры модели SAC (все остальное)
MODEL_KWARGS = {
    "buffer_size": 100000,
    "batch_size": 256,
    "learning_rate": 0.0003,
    "gamma": 0.99,
    "tau": 0.005,
    "ent_coef": 'auto',
    "learning_starts": 10000
    # policy_kwargs УБРАН ОТСЮДА
}


model_sac = agent.get_model(
    "sac",
    policy_kwargs=POLICY_KWARGS, 
    model_kwargs=MODEL_KWARGS   
)


trained_sac = agent.train_model(
    model=model_sac,
    tb_log_name='sac',
    total_timesteps=100000
)


trained_sac.save('./trained_models/trained_sac.zip')

print("Модель SAC успешно создана, обучена и сохранена.")

## Trading

In [38]:
# https://github.com/Musonda2day/Asset-Portfolio-Management-usingDeep-Reinforcement-Learning-/blob/main/8.%20DRL_Portfolios.ipynb

In [33]:
# https://github.com/rathiromil13/DS-5500-Project-Portfolio-Optimization-Using-Deep-Reinforcement-Learning/blob/master/documents/Final_Report_Portfolio_Optimization_using_Deep_Reinforcement_Learning.pdf

In [34]:
import os
import pandas as pd

# Конфигурация
DATA_DIR = "../data"
MODELS_PREDICTION_DIR = os.path.join(DATA_DIR, "models_predictions")

# Создание папки для сохранения результатов
os.makedirs(MODELS_PREDICTION_DIR, exist_ok=True)

def evaluate_and_save_models(models, df, env_kwargs):
    """
    Оценивает модели и сохраняет результаты в CSV файлы.

    Параметры:
        models (dict): Словарь с моделями, где ключ - название модели, значение - объект модели.
        df (pd.DataFrame): Исходный DataFrame с данными.
        env_kwargs (dict): Параметры для создания среды StockPortfolioEnv.

    Возвращает:
        None
    """
    # Разделение данных на тестовый период
    trade_data = data_split(df, start_test, end_test)
    print(f"Тестовые данные: {trade_data.shape}")

    for model_name, model in models.items():
        print(f"Обработка модели: {model_name}")
        try:
            # Создание среды для торговли
            trade_env = StockPortfolioEnv(df=trade_data, **env_kwargs)

            # Получение предсказаний модели
            df_daily_return, df_actions = DRLAgent.DRL_prediction(
                model=model,
                environment=trade_env
            )

            # Убедимся, что столбец 'date' имеет формат datetime
            if 'date' in df_daily_return.columns:
                df_daily_return['date'] = pd.to_datetime(df_daily_return['date'])

            # Сохранение результатов
            save_model_predictions(
                df_daily_return=df_daily_return,
                df_actions=df_actions,
                model_name=model_name,
                data_dir=DATA_DIR
            )

        except Exception as e:
            print(f"Ошибка при обработке модели {model_name}: {e}")

def save_model_predictions(df_daily_return, df_actions, model_name, data_dir):
    """
    Сохраняет результаты предсказаний модели (доходность и действия) в CSV файлы.

    Параметры:
        df_daily_return (pd.DataFrame): DataFrame с ежедневной доходностью.
        df_actions (pd.DataFrame): DataFrame с действиями модели.
        model_name (str): Название модели (используется в имени файла).
        data_dir (str): Путь к папке data.

    Возвращает:
        None
    """
    # Путь к папке для сохранения файлов
    models_prediction_dir = os.path.join(data_dir, "models_predictions")

    # Имена файлов
    return_file_name = os.path.join(models_prediction_dir, f"{model_name}_return_series.csv")
    actions_file_name = os.path.join(models_prediction_dir, f"{model_name}_actions.csv")

    try:
        # Сохранение DataFrame с ежедневной доходностью
        df_daily_return.to_csv(return_file_name, index=False)
        print(f"Файл '{return_file_name}' успешно сохранен.")

        # Сохранение DataFrame с действиями модели
        df_actions.to_csv(actions_file_name, index=True)
        print(f"Файл '{actions_file_name}' успешно сохранен.")

    except Exception as e:
        print(f"Ошибка при сохранении файлов для модели {model_name}: {e}")

In [36]:
models = {
    "model_a2c": model_a2c,
    # "model_ppo": model_ppo,
    # "model_ddpg": model_ddpg,
    # "model_sac": model_sac
}


# Вызов функции для оценки и сохранения результатов моделей
evaluate_and_save_models(models=models, df=df, env_kwargs=env_kwargs)

Тестовые данные: (14679, 96)
Обработка модели: model_a2c
begin_total_asset:1000000
end_total_asset:783542.1444604354
Sharpe:  -0.3851320473012059
hit end!
Файл '../data\models_predictions\model_a2c_return_series.csv' успешно сохранен.
Файл '../data\models_predictions\model_a2c_actions.csv' успешно сохранен.


In [37]:
import os
import pandas as pd
from stable_baselines3 import SAC
# from stable_baselines3.common.vec_env import DummyVecEnv # Не строго обязателен для load, но может быть нужен DRLAgent

# --- ШАГ 0: Убедитесь, что ВСЕ эти компоненты определены/импортированы ВЫШЕ ---

# 0.1. Ваш класс среды
# from your_module import StockPortfolioEnv
# class StockPortfolioEnv(gym.Env): ... (полное определение класса)

# 0.2. Функция разделения данных
# from your_module import data_split
# def data_split(df, start, end):
#     """
#     ВАЖНО: Убедитесь, что эта функция возвращает DataFrame trade_data
#     с индексом, который ожидает StockPortfolioEnv (вероятно, сброшенный
#     индекс 0, 1, 2... для дней внутри тестового периода).
#     Пример реализации из предыдущего ответа:
#     """
#     print(f"Разделение данных с {start} по {end}")
#     # Убедимся, что индекс df содержит дату (может быть MultiIndex)
#     if isinstance(df.index, pd.MultiIndex):
#         data = df[(df.index.get_level_values('date') >= start) & (df.index.get_level_values('date') < end)].copy()
#     else: # Если индекс - просто дата
#         data = df[(df.index >= start) & (df.index < end)].copy()
#
#     if data.empty:
#         raise ValueError(f"Нет данных для тестового периода {start} - {end}")
#
#     # Сбросим существующий индекс (date, tic или просто date)
#     data = data.reset_index()
#
#     # Переиндексируем дни 0, 1, 2... для среды StockPortfolioEnv
#     unique_dates = data['date'].unique()
#     date_to_day_map = {date: i for i, date in enumerate(unique_dates)}
#     data['day_index'] = data['date'].map(date_to_day_map)
#
#     # Установим новый индекс и отсортируем
#     # ВАЖНО: Проверьте, ожидает ли StockPortfolioEnv 'day_index' или просто числовой индекс
#     # Если используется df.loc[self.day, :], то 'day_index' как индекс идеален
#     data = data.set_index('day_index').sort_index()
#     print(f"Размер тестовых данных после обработки: {data.shape}")
#     print(f"Уникальных дней в тесте: {len(unique_dates)}")
#     return data

# 0.3. Функции оценки и сохранения
# def evaluate_and_save_models(models, df, env_kwargs, start_test, end_test, data_split_func): ... (ваш код)
# def save_model_predictions(df_daily_return, df_actions, model_name, data_dir): ... (ваш код)

# 0.4. Класс DRLAgent (если evaluate_and_save_models его использует)
# from finrl.agents.stablebaselines3.models import DRLAgent

# 0.5. Переменные
# df = pd.read_pickle("path/to/your/processed_dataframe.pkl") # Загрузите ПОЛНЫЙ датафрейм
# stock_dimension = df['tic'].nunique() # Определите stock_dimension
# state_space = stock_dimension # Определите state_space (убедитесь, что он правильный!)
# indicators = ['list', 'of', 'your', 'indicators'] # Определите список индикаторов
# DATA_DIR = "../data"
# MODELS_PREDICTION_DIR = os.path.join(DATA_DIR, "models_predictions")
# os.makedirs(MODELS_PREDICTION_DIR, exist_ok=True)

# # Даты теста
# start_test = pd.Timestamp("2022-01-01") # Используйте Timestamp для надежности
# end_test = pd.Timestamp("2023-01-01")

# # Параметры среды (ДОЛЖНЫ СОВПАДАТЬ с параметрами обучения!)
# env_kwargs = {
#     "hmax": 100,
#     "initial_amount": 1000000,
#     "transaction_cost_pct": 0.001,
#     "state_space": state_space, # Убедитесь, что state_space определен выше
#     "stock_dim": stock_dimension, # Убедитесь, что stock_dimension определен выше
#     "tech_indicator_list": indicators, # Убедитесь, что indicators определен выше
#     "action_space": stock_dimension,
#     "reward_scaling": 1e-4,
#     # "turbulence_threshold": None, # Добавьте, если использовали при обучении
#     # "lookback": ..., # Добавьте, если использовали при обучении
# }

# Путь к модели
model_path = './trained_models/trained_sac.zip'
# --- КОНЕЦ БЛОКА НЕОБХОДИМЫХ ОПРЕДЕЛЕНИЙ ---


# --- ШАГ 1: Загрузка Модели ---
print(f"Загрузка модели из: {model_path}")
try:
    # Загрузка модели. Передача env не обязательна, т.к. SB3 сохраняет структуру.
    loaded_model_sac = SAC.load(model_path)
    print(f"Модель SAC успешно загружена: {type(loaded_model_sac)}")

    # --- ШАГ 2: Подготовка к Оценке ---
    models_to_evaluate = {
        "SAC_Trained": loaded_model_sac # Даем модели осмысленное имя для сохранения файлов
        # Сюда можно добавить другие модели для сравнения
        # "DDPG_Trained": SAC.load('./trained_models/trained_ddpg.zip'),
    }

    # --- ШАГ 3: Запуск Оценки ---
    print("\nНачало оценки загруженной модели...")
    # Передаем все необходимые аргументы в вашу функцию
    evaluate_and_save_models(
        models=models_to_evaluate,
        df=df,                     # Полный датафрейм
        env_kwargs=env_kwargs,     # Параметры среды (те же, что при обучении)
        # Передаем start_test, end_test и data_split в функцию оценки, если они нужны там
        # Если evaluate_and_save_models сама вызывает data_split, то:
        # start_test=start_test,
        # end_test=end_test,
        # data_split_func=data_split # Если функция передается как аргумент
    )

    print("\nОценка завершена. Результаты должны быть сохранены в:", MODELS_PREDICTION_DIR)

# --- ШАГ 4: Обработка Ошибок ---
except FileNotFoundError:
    print(f"ОШИБКА: Файл модели не найден по пути {model_path}")
except NameError as e:
    print(f"\nОШИБКА: Не определена переменная или функция: {e}")
    print("--- Пожалуйста, убедитесь, что ВСЕ компоненты из 'ШАГ 0' определены и импортированы ПЕРЕД этим кодом! ---")
    print("--- Проверьте наличие: df, start_test, end_test, env_kwargs, StockPortfolioEnv, data_split, evaluate_and_save_models, save_model_predictions, DRLAgent (если используется) ---")
except ValueError as e:
     print(f"\nОШИБКА значения (возможно, в data_split или внутри среды): {e}")
except Exception as e:
    print(f"\nНеожиданная ОШИБКА при загрузке или оценке модели: {e}")
    import traceback
    traceback.print_exc() # Печатаем полный traceback для диагностики

Загрузка модели из: ./trained_models/trained_sac.zip
Модель SAC успешно загружена: <class 'stable_baselines3.sac.sac.SAC'>

Начало оценки загруженной модели...
Тестовые данные: (14679, 96)
Обработка модели: SAC_Trained
begin_total_asset:1000000
end_total_asset:803898.4505862627
Sharpe:  -0.28267288170641397
hit end!
Файл '../data\models_predictions\SAC_Trained_return_series.csv' успешно сохранен.
Файл '../data\models_predictions\SAC_Trained_actions.csv' успешно сохранен.

Оценка завершена. Результаты должны быть сохранены в: ../data\models_predictions
